# Import Libraries

In [2]:
#Import Libraries
import pandas as pd
import psycopg2

In [3]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 25)
pd.set_option('display.width', 1000)

# Connect to Database

In [5]:
#Connects to postgress db with your password
conn = psycopg2.connect(host="localhost",dbname="postgres",user="postgres",password="Scotty555!", port=5432)

#Must open cursor to write sql code.
cur = conn.cursor()

# Data Processing

### Clean up Database

In [8]:
#Deletes the blank countries from table
cur.execute("""DELETE FROM Public."01_serial_killer_data" AS t1
               WHERE t1."country" is null;
""")

#Commits changes to the database
conn.commit()

#Removes extra spaces from country data set.
cur.execute("""UPDATE Public."01_country_data"
               SET "country" =  regexp_replace("country", '\s+$', '');
""")

#Commits changes to the database
conn.commit()

#Creates the bursted standard country data
cur.execute("""      
               CREATE TEMP TABLE temp_standard_countries AS
               SELECT *
               FROM  Public."01_standard_countries" AS s1
               CROSS  JOIN LATERAL string_to_table(standard_country, '|') AS s(bursted_country)
               """)

#Commits changes to the database
conn.commit()

<>:10: SyntaxWarning: invalid escape sequence '\s'
<>:10: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Brian\AppData\Local\Temp\ipykernel_16984\1654041521.py:10: SyntaxWarning: invalid escape sequence '\s'
  cur.execute("""UPDATE Public."01_country_data"


### Creates Temp tables with data joined

In [10]:
#uses temp table of standard country data and creates clean combine table
cur.execute("""
               
              CREATE TEMP TABLE temp_combine_dataset AS
               SELECT 
                     trim(name) as killer_name
                   , trim(bursted_country) as country
                   ,left(start_year,4) as start_year
                   ,CASE
                   WHEN end_year is null THEN left(start_year,4)
                   ELSE
                   left(end_year,4) 
                   END as end_year
                   ,trim(left(possible_victims_low,4)) as possible_victims_low
                   ,trim(left(possible_victims_high,4)) as possible_victims_high
                   ,round(cast(violent_crimes_per_100k AS DECIMAL),2) as violent_crimes_per_100k
                   ,trim(left(crime_count,10)) as crime_count
    			   , trim(continent) as continent
    			   , trim(sub_continent) as sub_continent
               FROM temp_standard_countries
                   INNER JOIN Public."01_serial_killer_data" as k1
                       ON K1."country" = temp_standard_countries."standard_country"
                   INNER JOIN Public."01_standard_countries" as s1
                       ON k1."country" = s1."source_country"
    			   INNER JOIN  Public."01_country_data"    as c1
        			   ON c1."country" = "bursted_country"   
""")

# Data Conclusions

In [12]:
#Do your data processing here.
cur.execute("""
               
              
               
               SELECT * 
               FROM  "temp_combine_dataset"
               WHERE possible_victims_low is not null and 
               start_year ~'^[0-9]' and 
               end_year ~'^[0-9]' and 
               possible_victims_low ~'[0-9]$' and 
               possible_victims_high ~'^[0-9]$'
               
               
                             
""")

#Assigns returned query to field
data = cur.fetchall()

#Converts variable to data frame
df = pd.DataFrame(data)

#Updates headers
headers = ['killer_name', 'country','start_year','end_year','average_low_victims','average_high_victims','violent_crimes_per_100k','crime_count','continent','sub_continent']
df.columns = headers

#When your done you have to close the cursor and the connection.
cur.close()
conn.close()


In [13]:
#Converts the data types
df['violent_crimes_per_100k'] = df['violent_crimes_per_100k'].astype(str).astype(float)
df[['start_year', 'end_year','average_low_victims','average_high_victims','crime_count']] = df[['start_year', 'end_year','average_low_victims','average_high_victims','crime_count']].astype(str).astype(int)

#Creates difference column
df['years_active'] = df['end_year'] - df['start_year'] 

df = df[['killer_name','continent','sub_continent','country','average_low_victims','average_high_victims','violent_crimes_per_100k','crime_count','years_active']]

df

,killer_name,continent,sub_continent,country,average_low_victims,average_high_victims,violent_crimes_per_100k,crime_count,years_active
0,Sergei Martynov,Europe,Eastern Europe,Russia,9,9,6.80,9866,18
1,Sergei Martynov,Europe,Eastern Europe,Russia,9,9,6.80,9866,18
2,Lloyd Gomez,Americas,Northern America,United States,9,9,6.38,21594,1
3,Lloyd Gomez,Americas,Northern America,United States,9,9,6.38,21594,1
4,Timothy Krajcir,Americas,Northern America,United States,9,9,6.38,21594,0
5,Timothy Krajcir,Americas,Northern America,United States,9,9,6.38,21594,0
6,Frankford Slasher,Americas,Northern America,United States,9,9,6.38,21594,5
7,Frankford Slasher,Americas,Northern America,United States,9,9,6.38,21594,5
8,Kendall Francois,Americas,Northern America,United States,9,9,6.38,21594,2
9,Kendall Francois,Americas,Northern America,United States,9,9,6.38,21594,2


In [14]:
# look at table of victims per continent and sub continent, solve rate for each continent and sub_continent. Look at victims per killer at each. Look at average active length for each.

df.to_csv(r"C:\Users\Brian\02_serial_killer_combine_data.csv",index=False)